In [1]:
#langchain for lang pormpt
!pip install langchain_community
#Embedding
!pip install langchain_openai
!pip install langchain-cohere
#Vector Database
!pip install Chroma
!pip install chromadb
#Read pdf file
!pip install pymupdf
#model
!pip install langchain-google-genai
!pip install langchain-groq

In [2]:
import os
import sys
# Import classes from modules
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_cohere import CohereEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [3]:
# Generate API KEY from Gemini and OpenAI website and define as a variable.
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEYS"
os.environ["OPENAI_API_KEY"] = "sk-proj-Mgc"
os.environ["GROQ_API_KEY"] = "gsk_L5UmgN6KP"
os.environ["COHERE_API_KEY"] = "Es69gVjbJlEm0I"

In [4]:
# Define the path to your local PDF file
local_path = "/content/ปลา.pdf"

# Load and split the PDF file
if local_path:
    loader = PyMuPDFLoader(file_path=local_path)
    data = loader.load_and_split()
else:
    print("Upload a PDF file")
    sys.exit()

In [5]:
data

[Document(metadata={'source': '/content/ปลา.pdf', 'file_path': '/content/ปลา.pdf', 'page': 0, 'total_pages': 18, 'format': 'PDF 1.7', 'title': '', 'author': 'DOF', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2016', 'producer': 'Microsoft® Word 2016', 'creationDate': "D:20210629102312+07'00'", 'modDate': "D:20210629102443+07'00'", 'trapped': ''}, page_content='ประมาณการผลผลิตและมูลค่าสัตว์น้ำจากการประมงของประเทศไทย  \nพ.ศ. 2564 - 2566 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nกลุ่มสถิติการประมง กองนโยบายและแผนพัฒนาการประมง'),
 Document(metadata={'source': '/content/ปลา.pdf', 'file_path': '/content/ปลา.pdf', 'page': 1, 'total_pages': 18, 'format': 'PDF 1.7', 'title': '', 'author': 'DOF', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2016', 'producer': 'Microsoft® Word 2016', 'creationDate': "D:20210629102312+07'00'", 'modDate': "D:20210629102443+07'00'", 'trapped': ''}, page_content='สารบัญ \n \nเรื่อง  \n \n \n \n \n \n \n \n \n \n \nหน้า 

In [6]:
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=1024, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [7]:
texts

[Document(metadata={'source': '/content/ปลา.pdf', 'file_path': '/content/ปลา.pdf', 'page': 0, 'total_pages': 18, 'format': 'PDF 1.7', 'title': '', 'author': 'DOF', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2016', 'producer': 'Microsoft® Word 2016', 'creationDate': "D:20210629102312+07'00'", 'modDate': "D:20210629102443+07'00'", 'trapped': ''}, page_content='ประมาณการผลผลิตและมูลค่าสัตว์น้ำจากการประมงของประเทศไทย  \nพ.ศ. 2564 - 2566 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nกลุ่มสถิติการประมง กองนโยบายและแผนพัฒนาการประมง'),
 Document(metadata={'source': '/content/ปลา.pdf', 'file_path': '/content/ปลา.pdf', 'page': 1, 'total_pages': 18, 'format': 'PDF 1.7', 'title': '', 'author': 'DOF', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2016', 'producer': 'Microsoft® Word 2016', 'creationDate': "D:20210629102312+07'00'", 'modDate': "D:20210629102443+07'00'", 'trapped': ''}, page_content='สารบัญ \n \nเรื่อง  \n \n \n \n \n \n \n \n \n \n \nหน้า 

In [8]:
!pip install sentence-transformers

In [24]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma

# Initialize the HuggingFaceBgeEmbeddings
model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


In [10]:
# This part is used for embedding the docs and storing them into Vector DB and initializing the retriever.
embeddings = HuggingFaceBgeEmbeddings()
# embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

In [16]:
custom_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are an AI thai researcher model assistant respone in thai.
    You are an expert at answering questions about fish.
    You are given the following extracted parts of a long document and a question
    Answer the question based ONLY on the following context.


    {context}


    Original question: {question}""",
)

In [19]:
# Create the language model
llm1 = ChatGroq(model="llama3-70b-8192")

chain = ConversationalRetrievalChain.from_llm(
    llm=llm1,
    retriever=docsearch.as_retriever(),
    combine_docs_chain_kwargs={"prompt": custom_template}
)

In [ ]:
chat_history = []
query = None  # Initialize query to avoid potential reference error

while True:
    if not query:
        query = input("User: ")
    if query in ['quit', 'q', 'exit']:
        break
    result = chain.invoke({"question": query, "chat_history": chat_history})
    print("Chatbot:", result['answer'])

    chat_history.append((query, result['answer']))
    query = None

In [21]:
chat_history

[('สรุปปลาน้ำจืด',
  'จากข้อมูลที่ให้มา สามารถสรุปได้ว่า ปลาน้ำจืดมีผลผลิตเฉลี่ย 46,756 ตันต่อปี คิดเป็นมูลค่า 5,202 ล้านบาทต่อปี โดยปลากะพงเป็นชนิดปลาที่มีผลผลิตสูงสุด คิดเป็นร้อยละ 96.65 ของผลผลิตปลาน้ำจืดทั้งหมด'),
 ('สรุปข้อความส่่วนสำคัญ',
  'ข้อความสำคัญส่วนสำคัญคือ ผลผลิตและมูลค่าสัตว์น้ำจากการเพาะเลี้ยงสัตว์น้ำจืดและชายฝั่งจะมีแนวโน้มเพิ่มขึ้นในอัตราร้อยละ 1.42 และ 3.40 ต่อปี ตามลำดับ เนื่องจากความต้องการบริโภคสัตว์น้ำยังคงเพิ่มขึ้น ประกอบกับภาครัฐมีแผนพัฒนาและส่งเสริมการเพาะเลี้ยงสัตว์น้ำทั้งทางด้านศักยภาพการผลิตและการตลาดภายในประเทศและส่งเสริมสร้างโอกาสการแข่งขันในตลาดต่างประเทศเพื่อให้การเพาะเลี้ยงสัตว์น้ำมีความยั่งยืน')]